# County Level

In [71]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
import statsmodels.api as sm 
import os
from functools import reduce

We focus on data in 2017 

### Median ZHVI

In [10]:
path = '/Users/liusulin/Desktop/Data Science/Project2/'

In [137]:
median_zhvi = pd.read_csv(path + 'County_MedianValuePerSqft_AllHomes.csv',
                         encoding = "ISO-8859-1").dropna()
# len(median_zhvi['RegionName'].unique()
# median_zhvi.drop_duplicates()
#calcualte return
median_zhvi['return'] = 100 * (median_zhvi['2017-12']-median_zhvi['2016-12'])/median_zhvi['2016-12']

In [138]:
median_zhvi['return']

0        7.671958
1        4.545455
2        5.617978
3        7.352941
4        7.854985
5        5.958549
6        8.143939
7        8.139535
8       14.432990
9        6.696429
10       7.142857
11       8.717949
12      13.600000
13      15.460526
14      18.181818
15      11.458333
16      15.814696
17       8.125000
18       7.526882
20      12.121212
21      10.931174
22       7.746479
23       7.395498
24       9.547739
25       6.751055
26       7.023411
27       6.578947
28       7.594937
29       9.734513
30       8.333333
          ...    
1442    15.068493
1443     9.523810
1450     5.660377
1453     2.702703
1454     5.128205
1459    11.940299
1462     8.235294
1465    12.820513
1467     3.932584
1468     6.557377
1477     9.803922
1490     3.759398
1494     3.571429
1501     8.771930
1506     8.910891
1507     7.258065
1511    11.242604
1533    11.304348
1537    11.718750
1548     6.250000
1551     9.278351
1561     6.944444
1562     4.081633
1567     2.439024
1574     4

### Earnings

In [106]:
earnings = pd.read_excel(path + 'EAN01.xls',sheet_name = 'Sheet1').dropna()

### Education

In [107]:
edu = pd.read_excel(path + 'EDU01.xls',sheet_name = 'Sheet1').dropna()

### Forecast

In [108]:
forecast = pd.read_csv(path + 'AllRegionsForePublic.csv',encoding = "ISO-8859-1")

In [139]:
dfs = [earnings, edu, forecast, median_zhvi]
df_final = reduce(lambda left,right: pd.merge(left,right,on='RegionName', how = 'left'), dfs)

In [140]:
df_final.dropna(subset = ['return','EAN010201D',
          'ForecastYoYPctChange',
          'EDU010187D'], inplace = True)

In [141]:
df_final[['return','EAN010201D',
          'ForecastYoYPctChange',
          'EDU010187D']] = preprocessing.scale(df_final[['return',
                                                         'EAN010201D',
                                                         'ForecastYoYPctChange',
                                                         'EDU010187D']])


### OLS Regression

In [208]:
np.random.seed(42)
index = np.random.randint(nsample * 3, size = nsample)
index

array([102, 270, 106,  71, 188,  20, 102, 121, 214,  87,  99, 151, 130,
       149, 257, 293, 191, 276, 160,  21, 252, 235,  48,  58, 169, 187,
       270, 189, 174,  50,  54, 243, 130, 134,  20, 166, 273,  88,  13,
       241, 264,  52,  91, 263,  34, 205,  80,  49,   1,  53, 105, 259,
       190, 217,  43, 161, 201, 269, 270, 214, 251, 189, 295, 212, 207,
       236,  52, 279, 216, 251, 187,  40, 156,  14,  64,   8, 128, 135,
        62, 138,  80, 162, 288, 260, 230,  40,  27, 134, 200, 267,  32,
        47,  61, 215, 292,  98, 171, 213,  34, 226])

In [210]:
df_final[['EDU010187D','EAN010201D']].iloc[index,:]

,EDU010187D,EAN010201D
105,0.562167,-0.013208
275,-0.161136,-0.328177
109,-0.407200,-0.013208
74,-0.493244,-0.013208
193,-0.408997,-0.328177
23,-0.333094,-0.314333
105,0.562167,-0.013208
124,-0.338015,-0.013208
219,-0.370533,-0.328177
90,-0.502357,-0.013208


In [214]:

y = df_final['return'][:1000]
x = df_final[['EDU010187D','EAN010201D']][:1000]
# y.shape
# x.shape
models = sm.OLS(y, x)
result = models.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 return   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     41.77
Date:                Wed, 05 Dec 2018   Prob (F-statistic):           3.78e-18
Time:                        22:12:43   Log-Likelihood:                -1616.6
No. Observations:                1000   AIC:                             3237.
Df Residuals:                     998   BIC:                             3247.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
EDU010187D     0.0057      0.016      0.350      0.727      -0.026       0.038
EAN010201D     1.1517      0.126      9.133      0.000       0.904       1.399
==============================================================================
Omnibus:                       18.016   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.700
Skew:                          -0.328   Prob(JB):                     8.70e-05
Kurtosis:                       2.861   Cond. No.                         7.71
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Cross sectional: we found that education is not significant but earnings are significant. We could use earning to predict house prices.